In [2]:
import os
import cv2
from PIL import Image
import numpy as np
import pandas as pd

## Data load

In [3]:
base_dir = '/root/MZC_Yubin/Sorin'
data_dir = [os.path.join(base_dir, 'dataset_220524'), os.path.join(base_dir, 'dataset_220525')]
# labels = ['ETC', 'Hole', 'Swell_Nomal', 'Swell_pointed','Swoot', 'Bad']
labels = ['Hole', 'Swell_Nomal', 'Swell_pointed','Swoot']

data_list = []
label_list = []

for direct in data_dir:
    for label in labels:
        if label != 'Bad':
            for path in os.listdir(os.path.join(direct,f'B_{label}')):
                if 'ipynb_checkpoints' in path:
                    pass
                else:
                    data_list.append(os.path.join(direct,f'B_{label}',path))
                    if 'Swell' in label:
                        label_list.append('Swell')
                    else:
                        label_list.append(label)
        else:
            for path in os.listdir(os.path.join(direct, label)):
                if 'ipynb_checkpoints' in path:
                    pass
                else:
                    data_list.append(os.path.join(direct,label,path))
                    label_list.append(label)
                    
# for direct in data_dir:
#     for label in labels:
#         if label != 'Bad':
#             for path in os.listdir(os.path.join(direct,f'B_{label}')):
#                 if 'ipynb_checkpoints' in path:
#                     pass
#                 else:
#                     data_list.append(os.path.join(direct,f'B_{label}',path))
#                     label_list.append(label)
#         else:
#             for path in os.listdir(os.path.join(direct, label)):
#                 if 'ipynb_checkpoints' in path:
#                     pass
#                 else:
#                     data_list.append(os.path.join(direct,label,path))
#                     label_list.append(label)

In [4]:
img_num = len(data_list)
n = 1
img_list = []
for i in data_list[:img_num]:
    if n%50 == 0:
        print(n)
    img = Image.open(i)
    image_resize = img.resize((224,224))
    img_list.append(np.array(image_resize))
    n+=1
    
img_array = np.array(img_list)

50
100
150


In [5]:
temp_label = label_list[:img_num]

label_df = pd.DataFrame(columns = ['label'], data = temp_label)
label_df = pd.get_dummies(label_df['label'])
label_df.shape

(196, 3)

## Custom Modeling

In [6]:
from sklearn.metrics import confusion_matrix,f1_score, accuracy_score, precision_score, recall_score
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout, MaxPooling2D, Input, Conv2DTranspose, Concatenate, Resizing, BatchNormalization
from tensorflow.keras import backend as K 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from sklearn.metrics import f1_score, accuracy_score

In [7]:
# label = list(set(label_list))

# K.clear_session()
# model = Sequential()
# model.add(Conv2D(3, 32, padding = 'valid', activation = 'relu', input_shape = img_array.shape[1:]))
# model.add(Conv2D(1, 32, padding = 'valid', activation = 'relu'))
# model.add(MaxPooling2D(8))
# model.add(Conv2D(1, 6, padding = 'valid', activation = 'relu'))
# model.add(Flatten())
# model.add(Dense(32, activation = 'relu'))
# model.add(Dense(len(list(label_df)), activation = 'softmax'))
# model.summary()

# early_stopping_callback = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10,restore_best_weights=True, mode='min')
# # model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

# model.fit(img_array, label_df, epochs = 100, callbacks = early_stopping_callback, batch_size = 8, validation_split=0.1, verbose = 1)

In [8]:
# print(f1_score(np.array(label_df).argmax(1), model.predict(img_array).argmax(1), average='macro'))
# print(accuracy_score(np.array(label_df).argmax(1), model.predict(img_array).argmax(1)))

## Check Feature map

In [9]:
# import matplotlib.pyplot as plt

# img = img_array[30]
# img = img.reshape(-1,520,520,3)

# temp = keras.Model(inputs=model.inputs, outputs=model.layers[0].output)
# feature_maps = temp.predict(img)

# # plot all 64 maps in an 8x8 squares
# square_x = 3
# square_y = 1
# ix = 1
# plt.figure(figsize=(15, 10)) 

# for _ in range(square_x):
#     for _ in range(square_y):
#         ax = plt.subplot(square_x, square_y, ix)
#         ax.set_xticks([])
#         ax.set_yticks([])
#         # plot filter channel in grayscale
#         plt.imshow(feature_maps[0, :, :, ix-1], cmap='gray')
#         ix += 1
# plt.show()

## ResNet 101

In [10]:
from keras.applications.resnet import ResNet101

K.clear_session()

base_model = ResNet101(include_top = False, weights = "imagenet", input_shape = img_array.shape[1:])

resize_model = Sequential()
resize_model.add(Input(shape = img_array.shape[1:]))
# resize_model.add(Resizing(350,350, interpolation = 'gaussian'))
# resize_model.add(Conv2D(3,3,padding = 'same', activation = 'relu'))
resize_model.add(base_model)
resize_model.add(Flatten())
resize_model.add(Dropout(0.5))
resize_model.add(Dense(len(list(label_df)), activation = 'softmax'))
resize_model.summary()

resize_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101 (Functional)       (None, 7, 7, 2048)        42658176  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dropout (Dropout)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 3)                 301059    
Total params: 42,959,235
Trainable params: 42,853,891
Non-trainable params: 105,344
_________________________________________________________________


In [11]:
early_stopping_callback = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10,restore_best_weights=True, mode='min')
resize_model.fit(img_array, label_df, epochs = 100, callbacks = early_stopping_callback, batch_size = 8, validation_split=0.1, verbose = 1)

Epoch 1/100
Extension horovod.torch has not been built: /usr/local/lib/python3.8/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-38-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2022-09-15 04:42:04.537 tensorflow-2-6-gpu--ml-g4dn-xlarge-7eba0e151157af0bd8a25c896f03:48 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-09-15 04:42:04.564 tensorflow-2-6-gpu--ml-g4dn-xlarge-7eba0e151157af0bd8a25c896f03:48 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
22/22 [==============================] - 21s 278ms/step - loss: 8.1365 - accuracy: 0.5000 - val_loss: 88419648.0000 - val_accuracy: 0.4000
Epoch 2/100
22/22 [==============================] - 3s 159ms/step - loss: 5.1888 - accuracy: 0.5398 - val_loss: 942844288.0000 - val_accuracy: 0.6000
Epoch 3/

In [12]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix

print(f1_score(np.array(label_df).argmax(1), resize_model.predict(img_array).argmax(1), average='macro'))
print(accuracy_score(np.array(label_df).argmax(1), resize_model.predict(img_array).argmax(1)))
confusion_matrix(np.array(label_df).argmax(1), resize_model.predict(img_array).argmax(1))

0.5940717675639493
0.6836734693877551


array([[ 9, 34, 19],
       [ 7, 42,  1],
       [ 1,  0, 83]])

## VGG 16

In [13]:
from tensorflow.python.keras.applications.vgg16 import VGG16

## VGG 모델 불러오기
K.clear_session()

base_model = VGG16(include_top = False, weights = "imagenet", input_shape = img_array.shape[1:])

resize_model = Sequential()
resize_model.add(Input(shape = img_array.shape[1:]))
# resize_model.add(Resizing(350,350, interpolation = 'gaussian'))
# resize_model.add(Conv2D(3,3,padding = 'same', activation = 'relu'))
resize_model.add(base_model)
resize_model.add(Flatten())
resize_model.add(Dropout(0.5))
resize_model.add(Dense(len(list(label_df)), activation = 'softmax'))
resize_model.summary()

resize_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

# model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper (ModuleWrappe (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dropout (Dropout)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 3)                 75267     
Total params: 14,789,955
Trainable params: 14,789,955
Non-trainable params: 0
_________________________________________________________________


In [14]:
early_stopping_callback = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10,restore_best_weights=True, mode='min')
resize_model.fit(img_array, label_df, epochs = 100, callbacks = early_stopping_callback, batch_size = 8, validation_split=0.1, verbose = 1)

Epoch 1/100
22/22 [==============================] - 16s 521ms/step - loss: 24.0662 - accuracy: 0.3864 - val_loss: 0.9243 - val_accuracy: 0.6000
Epoch 2/100
22/22 [==============================] - 3s 129ms/step - loss: 1.0248 - accuracy: 0.4375 - val_loss: 0.9669 - val_accuracy: 0.2000
Epoch 3/100
22/22 [==============================] - 3s 129ms/step - loss: 1.0342 - accuracy: 0.5625 - val_loss: 0.9265 - val_accuracy: 0.3000
Epoch 4/100
22/22 [==============================] - 3s 129ms/step - loss: 0.9926 - accuracy: 0.4943 - val_loss: 0.7836 - val_accuracy: 0.6000
Epoch 5/100
22/22 [==============================] - 3s 129ms/step - loss: 0.8218 - accuracy: 0.5625 - val_loss: 0.7751 - val_accuracy: 0.5500
Epoch 6/100
22/22 [==============================] - 3s 129ms/step - loss: 0.7604 - accuracy: 0.6420 - val_loss: 0.4386 - val_accuracy: 1.0000
Epoch 7/100
22/22 [==============================] - 3s 130ms/step - loss: 0.7754 - accuracy: 0.6534 - val_loss: 0.3603 - val_accuracy: 0.90

In [15]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix

print(f1_score(np.array(label_df).argmax(1), resize_model.predict(img_array).argmax(1), average='macro'))
print(accuracy_score(np.array(label_df).argmax(1), resize_model.predict(img_array).argmax(1)))
confusion_matrix(np.array(label_df).argmax(1), resize_model.predict(img_array).argmax(1))

0.558249346053239
0.6938775510204082


array([[ 2, 34, 26],
       [ 0, 50,  0],
       [ 0,  0, 84]])

In [43]:
# import matplotlib.pyplot as plt 

# plt.figure(figsize = (10,10))
# plt.imshow(img_array[0])
# plt.show()

In [44]:
resize_model.predict(img_array).argmax(1)

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [45]:
np.array(label_df).argmax(1)

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])